In [54]:
# import json
# import pandas as pd

# # Load the JSON file
# with open('data/telegram_full/result.json', encoding='utf-8') as f:
#     data = json.load(f)

# # Extract just the 'messages' list
# messages = data["messages"]

# # Load into a DataFrame
# df = pd.DataFrame(messages)

# df = df[df["action"].isna()]

# print(df["text"].dtypes)

# # df.to_csv("data/messages.csv")


In [55]:
from datasets import load_dataset

dataset = load_dataset('csv', data_files='data/messages.csv')
split_dataset = dataset['train'].train_test_split(test_size=0.2, shuffle=True, seed=1)

In [56]:
split_dataset["train"][0]

{'Unnamed: 0': 1913,
 'id': 1926,
 'type': 'message',
 'date': '2023-08-14T00:28:32',
 'date_unixtime': 1691965712,
 'actor': None,
 'actor_id': None,
 'action': None,
 'title': None,
 'text': 'ну сейчас может быть, но так не всегда было',
 'text_entities': "[{'type': 'plain', 'text': 'ну сейчас может быть, но так не всегда было'}]",
 'from': '🐹',
 'from_id': 'user378930791',
 'reply_to_message_id': 1923.0,
 'photo': None,
 'photo_file_size': None,
 'width': None,
 'height': None,
 'edited': None,
 'edited_unixtime': None,
 'reactions': None,
 'inviter': None,
 'members': None,
 'file': None,
 'file_name': None,
 'file_size': None,
 'thumbnail': None,
 'thumbnail_file_size': None,
 'mime_type': None,
 'location_information': None,
 'forwarded_from': None,
 'media_type': None,
 'sticker_emoji': None,
 'duration_seconds': None,
 'poll': None,
 'via_bot': None,
 'place_name': None,
 'address': None,
 'message_id': None}

In [57]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")

In [58]:
def preprocess_function(examples):
    return tokenizer(str(examples["text"]) if examples["text"] is not None else "")

In [59]:
tokenized_dataset = split_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=split_dataset["train"].column_names,
)

tokenized_dataset["test"][2]

Map: 100%|██████████| 1124/1124 [00:00<00:00, 7363.61 examples/s]


{'input_ids': 25, 'attention_mask': 1}

In [60]:
block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    return result

In [61]:
lm_dataset = tokenized_dataset.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/66475 [00:00<?, ? examples/s]


TypeError: can only concatenate list (not "int") to list

In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained("FacebookAI/xlm-roberta-base")

Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="my_awesome_mlm_model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

/tmp/ipykernel_28727/2567528350.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 2.58


In [ ]:
from transformers import pipeline

text = "Леша самый <mask> гей."

mask_filler = pipeline("fill-mask", "/home/sergei/Documents/pets/spambot/my_awesome_mlm_model/checkpoint-924")
mask_filler(text, top_k=3)

Device set to use cuda:0


[{'score': 0.5291919112205505,
  'token': 1437,
  'token_str': ' ',
  'sequence': 'Леша самый  гей.'},
 {'score': 0.18665103614330292,
  'token': 111,
  'token_str': ' -',
  'sequence': 'Леша самый - гей.'},
 {'score': 0.055373139679431915,
  'token': 17487,
  'token_str': ' ?',
  'sequence': 'Леша самый ? гей.'}]